In [0]:
# This file performs a black box adversrial attack on network used to differetiate malicious 
# from benign flows. It uses Sparse1D attacks to attack the network. In order to generate
# new examples for training it uses a Jacobian graph  

# Initialize drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
# Move to drive 
%cd drive
%cd 'My Drive'
%cd Oxford-Thesis

In [0]:
 import numpy as np
import tensorflow as tf
import six 
import math
from tensorflow.python.training import moving_averages 
import pickle
import tensorflow.keras.backend as K
from sklearn.preprocessing import normalize
import tensorflow_probability as tfp
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
import functools
import logging
import numpy as np
from six.moves import xrange
import tensorflow as tf
CONT_FEATURE_SIZE = 158

In [0]:
!pip install git+https://github.com/tensorflow/cleverhans.git#egg=cleverhans

In [0]:
from cleverhans.attacks import FastGradientMethod
from cleverhans.utils_tf import jacobian_graph, jacobian_augmentation
from cleverhans.compat import flags
from cleverhans.initializers import HeReLuNormalInitializer
from cleverhans.attacks import SparseL1Descent  
from cleverhans.loss import CrossEntropy
from cleverhans.model import Model
from cleverhans.train import train
from cleverhans.utils import set_log_level
from cleverhans.utils import TemporaryLogLevel
from cleverhans.utils import to_categorical
from cleverhans.utils_tf import model_eval
from cleverhans.evaluation import batch_eval
import pickle
import cleverhans.model as chm
import cleverhans as ch

In [0]:
## Constants for creating the black box substitute model 
NB_CLASSES = 2
LEARNING_RATE = .001
NB_EPOCHS = 10
HOLDOUT = 150
DATA_AUG = 6
NB_EPOCHS_S = 10
LMBDA = .1
AUG_BATCH_SIZE = 128

In [0]:
## Load in data for training the substitute model 
with open('./embedding/CNN_FULL/USTC_rn_mal_meta_valid', 'rb') as fp:
  mal_valid_meta_X = pickle.load(fp)
  fp.close()

with open('./embedding/CNN_FULL/USTC_rn_benign_meta_valid', 'rb') as fp:
  benign_valid_meta_X = pickle.load(fp)
  benign_valid_meta_X = np.array(benign_valid_meta_X)
  fp.close()
valid_labels = [[0,1]]*len(benign_valid_meta_X) +[[1,0]]*len(mal_valid_meta_X)

import numpy as np
valid_labels = [[0,1]]*len(benign_valid_meta_X) +[[1,0]]*len(mal_valid_meta_X)
valid_meta_X = np.concatenate([benign_valid_meta_X,mal_valid_meta_X])
c = list(zip(valid_meta_X,valid_labels))
np.random.shuffle(c)
valid_meta_X,valid_labels = zip(*c)
valid_X =np.array(valid_meta_X).astype(float)

In [0]:
## CNN model for differentiating the malicious and bengin data
def get_logits_cnn(x,return_embedding= False):
  with  tf.variable_scope("tree_second_model",reuse=tf.AUTO_REUSE):
    with  tf.device('/device:GPU:0'):
      W_dense4_class = tf.get_variable("dense4_sec",initializer =tf.random_normal(stddev=0.1, shape =[158,784] ))
      b_dense4_class = tf.get_variable("bias4_sec",initializer =tf.random_normal(stddev=0.1,shape=[784]))
      h_fc4_class = tf.nn.relu(tf.matmul(tf.cast(x,tf.float32), W_dense4_class) + b_dense4_class)

      reshape_1_class = tf.reshape(h_fc4_class,[BATCH_SHAPE,28,28,1])
      conv2d_1_weight = tf.get_variable("conv2d_1_dense",initializer =tf.random_normal(stddev=0.1, shape =[5, 5, 1, 32]))
      conv2d_1_bias = tf.get_variable("conv2d_1_bias",initializer =tf.random_normal(stddev=0.1, shape= [32]))
      conv2d_1_class = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(reshape_1_class,
                                                  conv2d_1_weight,
                                                  strides =[1,1,1,1],
                                                  padding='SAME'),
                                      conv2d_1_bias))

      maxpool2d_1_class = tf.nn.max_pool(conv2d_1_class,
                                       ksize=[1, 2, 2, 1], 
                                       strides=[1, 2, 2, 1],
                                padding='SAME')
      conv2d_2_weight = tf.get_variable("conv2d_2_dense",initializer =tf.random_normal(stddev=0.1, shape =[5, 5, 32, 64]))
      conv2d_2_bias = tf.get_variable("conv2d_2_bias",initializer =tf.random_normal(stddev=0.1, shape=[64]))
      conv2d_2_class = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(maxpool2d_1_class,
                                                  conv2d_2_weight,
                                                  strides =[1,1,1,1],
                                                  padding='SAME'),
                                      conv2d_2_bias))

      maxpool2d_2_class = tf.nn.max_pool(conv2d_2_class,
                                       ksize=[1, 2, 2, 1], 
                                       strides=[1, 2, 2, 1],
                                padding='SAME')

      reshape_2_class = tf.reshape(maxpool2d_2_class,[BATCH_SHAPE,-1])
      drop_one = tf.nn.dropout(reshape_2_class,rate=0.2)
      W_dense5_class = tf.get_variable("dense5_sec",initializer =tf.random_normal(stddev=0.1,shape=[7*7*64,1024]))
      b_dense5_class = tf.get_variable("bias5_sec",initializer =tf.random_normal(stddev=0.1,shape=[1024]))
      h_fc5_logit = tf.nn.bias_add(tf.matmul(drop_one, W_dense5_class),b_dense5_class)
      h_fc5_class = tf.nn.relu(h_fc5_logit)
      drop_two = tf.nn.dropout(h_fc5_class,rate=0.4)

      out_weight_class =  tf.get_variable("out_dense_sec",initializer =tf.random_normal(stddev=0.1,shape=[1024, 2]))
      out_bias_class =  tf.get_variable("out_bias_sec",initializer =tf.random_normal(stddev=0.1,shape = [2]))
      out_final_logits = tf.nn.bias_add(tf.matmul(drop_two,out_weight_class),out_bias_class)
      if return_embedding == True:
        return h_fc5_logit
      return out_final_logits

In [0]:
def get_logits(x):
  return get_logits_cnn(get_rep(x, full_rep=True ))

In [0]:
## Define the placeholders for training the cnn models 
 with tf.device('/device:GPU:0'):
  x2_ = tf.placeholder(tf.float32,shape= [BATCH_SHAPE,CONT_FEATURE_SIZE])
  y2_ =tf.placeholder(tf.float32,shape= [BATCH_SHAPE,2])

In [0]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
print("GPU Available: ", tf.test.is_gpu_available())
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
## Tensors for training the cnn model 
with tf.device('/device:GPU:0'):
  cnn_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y2_, logits=get_logits_cnn(x2_)))
  cnn_class_train_step = tf.train.AdamOptimizer(1e-4,name="class-adam-encoder").minimize(cnn_cross_entropy)
  cnn_correct_prediction = tf.equal(tf.argmax(get_logits_cnn(x2_), 1), tf.argmax(y2_, 1))
  cnn_accuracy = tf.reduce_mean(tf.cast(cnn_correct_prediction, tf.float32))

In [0]:
## Saver for saving and reloading the original cnn model and the black box model 
second_saver = tf.train.Saver()

In [0]:
## Sub logits for retraining the model after getting the substitute data
def get_next_sub_logits(x,return_embedding= False):
  with  tf.variable_scope("sub1",reuse=tf.AUTO_REUSE):
    with tf.device('/device:GPU:0'):
      W_dense4_class = tf.get_variable("dense4_sec",initializer =tf.random_normal(stddev=0.1, shape =[AUTO_ENCODER_OUT_SHAPE,784] ))
      b_dense4_class = tf.get_variable("bias4_sec",initializer =tf.random_normal(stddev=0.1,shape=[784]))
      h_fc4_class = tf.nn.relu(tf.matmul(tf.cast(x,tf.float32), W_dense4_class) + b_dense4_class)

      reshape_1_class = tf.reshape(h_fc4_class,[BATCH_SHAPE,28,28,1])
      conv2d_1_weight = tf.get_variable("conv2d_1_dense",initializer =tf.random_normal(stddev=0.1, shape =[5, 5, 1, 32]))
      conv2d_1_bias = tf.get_variable("conv2d_1_bias",initializer =tf.random_normal(stddev=0.1, shape= [32]))
      conv2d_1_class = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(reshape_1_class,
                                                  conv2d_1_weight,
                                                  strides =[1,1,1,1],
                                                  padding='SAME'),
                                      conv2d_1_bias))

      maxpool2d_1_class = tf.nn.max_pool(conv2d_1_class,
                                       ksize=[1, 2, 2, 1], 
                                       strides=[1, 2, 2, 1],
                                padding='SAME')
      conv2d_2_weight = tf.get_variable("conv2d_2_dense",initializer =tf.random_normal(stddev=0.1, shape =[5, 5, 32, 64]))
      conv2d_2_bias = tf.get_variable("conv2d_2_bias",initializer =tf.random_normal(stddev=0.1, shape=[64]))
      conv2d_2_class = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(maxpool2d_1_class,
                                                  conv2d_2_weight,
                                                  strides =[1,1,1,1],
                                                  padding='SAME'),
                                      conv2d_2_bias))

      maxpool2d_2_class = tf.nn.max_pool(conv2d_2_class,
                                       ksize=[1, 2, 2, 1], 
                                       strides=[1, 2, 2, 1],
                                padding='SAME')

      reshape_2_class = tf.reshape(maxpool2d_2_class,[BATCH_SHAPE,-1])
      drop_one = tf.nn.dropout(reshape_2_class,rate=0.2)
      W_dense5_class = tf.get_variable("dense5_sec",initializer =tf.random_normal(stddev=0.1,shape=[7*7*64,1024]))
      b_dense5_class = tf.get_variable("bias5_sec",initializer =tf.random_normal(stddev=0.1,shape=[1024]))
      h_fc5_logit = tf.nn.bias_add(tf.matmul(drop_one, W_dense5_class),b_dense5_class)
      h_fc5_class = tf.nn.relu(h_fc5_logit)
      drop_two = tf.nn.dropout(h_fc5_class,rate=0.4)

      out_weight_class =  tf.get_variable("out_dense_sec",initializer =tf.random_normal(stddev=0.1,shape=[1024, 2]))
      out_bias_class =  tf.get_variable("out_bias_sec",initializer =tf.random_normal(stddev=0.1,shape = [2]))
      out_final_logits = tf.nn.bias_add(tf.matmul(drop_two,out_weight_class),out_bias_class)
      if return_embedding == True:
        return h_fc5_logit
      return out_final_logits
    

In [0]:
## Sub logits for training the model while getting the substitute data
def get_sub_logits(x,return_embedding= False):
  with tf.device('/device:GPU:0'):
      W_dense4_class = tf.get_variable("dense4_sec",initializer =tf.random_normal(stddev=0.1, shape =[AUTO_ENCODER_OUT_SHAPE,784] ))
      b_dense4_class = tf.get_variable("bias4_sec",initializer =tf.random_normal(stddev=0.1,shape=[784]))
      h_fc4_class = tf.nn.relu(tf.matmul(tf.cast(x,tf.float32), W_dense4_class) + b_dense4_class)

      reshape_1_class = tf.reshape(h_fc4_class,[BATCH_SHAPE,28,28,1])
      conv2d_1_weight = tf.get_variable("conv2d_1_dense",initializer =tf.random_normal(stddev=0.1, shape =[5, 5, 1, 32]))
      conv2d_1_bias = tf.get_variable("conv2d_1_bias",initializer =tf.random_normal(stddev=0.1, shape= [32]))
      conv2d_1_class = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(reshape_1_class,
                                                  conv2d_1_weight,
                                                  strides =[1,1,1,1],
                                                  padding='SAME'),
                                      conv2d_1_bias))

      maxpool2d_1_class = tf.nn.max_pool(conv2d_1_class,
                                       ksize=[1, 2, 2, 1], 
                                       strides=[1, 2, 2, 1],
                                padding='SAME')
      conv2d_2_weight = tf.get_variable("conv2d_2_dense",initializer =tf.random_normal(stddev=0.1, shape =[5, 5, 32, 64]))
      conv2d_2_bias = tf.get_variable("conv2d_2_bias",initializer =tf.random_normal(stddev=0.1, shape=[64]))
      conv2d_2_class = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(maxpool2d_1_class,
                                                  conv2d_2_weight,
                                                  strides =[1,1,1,1],
                                                  padding='SAME'),
                                      conv2d_2_bias))

      maxpool2d_2_class = tf.nn.max_pool(conv2d_2_class,
                                       ksize=[1, 2, 2, 1], 
                                       strides=[1, 2, 2, 1],
                                padding='SAME')

      reshape_2_class = tf.reshape(maxpool2d_2_class,[BATCH_SHAPE,-1])
      drop_one = tf.nn.dropout(reshape_2_class,rate=0.2)
      W_dense5_class = tf.get_variable("dense5_sec",initializer =tf.random_normal(stddev=0.1,shape=[7*7*64,1024]))
      b_dense5_class = tf.get_variable("bias5_sec",initializer =tf.random_normal(stddev=0.1,shape=[1024]))
      h_fc5_logit = tf.nn.bias_add(tf.matmul(drop_one, W_dense5_class),b_dense5_class)
      h_fc5_class = tf.nn.relu(h_fc5_logit)
      drop_two = tf.nn.dropout(h_fc5_class,rate=0.4)

      out_weight_class =  tf.get_variable("out_dense_sec",initializer =tf.random_normal(stddev=0.1,shape=[1024, 2]))
      out_bias_class =  tf.get_variable("out_bias_sec",initializer =tf.random_normal(stddev=0.1,shape = [2]))
      out_final_logits = tf.nn.bias_add(tf.matmul(drop_two,out_weight_class),out_bias_class)
      if return_embedding == True:
        return h_fc5_logit
      return out_final_logits

In [0]:
## Variable scope for changing the scope while training the substitute model 
TOP_SCOPE = tf.get_variable_scope()
MODEL_SCOPE = tf.get_variable_scope()

In [0]:
## Gets accuracy based on true and predicted logits
def get_accuracy(true_logits,predicted_logits):
  accuracy = 0 
  for index in range(len(true_logits)):
    true_predict = np.argmax(true_logits[index])
    adv_predict = np.argmax(predicted_logits[index])
    if true_predict == adv_predict:
      accuracy +=1
  return float(accuracy)/len(true_logits)

In [0]:
## Prepares the black box for training 
def prep_bbox(sess):
  with  tf.device('/device:GPU:0'):
    nn_model = chm.CallableModelWrapper(get_logits_cnn, 'logits')
    print("Classification Network Benign vs Malicious ")
    batch = valid_X[:BATCH_SHAPE]
    print(len(batch))
    non_adv_output = get_logits_cnn(tf.cast(np.array(batch),tf.float32))
    valid_acc = 0
    for batch_num in range(1,int(len(valid_X)/BATCH_SHAPE)): 
      print(batch_num)
      batch = valid_X[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
      batch_label = valid_labels[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
      non_adv_out = get_logits_cnn(tf.cast(np.array(batch),tf.float32))
      non_adv_output = tf.concat([non_adv_output,non_adv_out],axis=0)
    valid_cnn_logits = sess.run(non_adv_output) 
    return nn_model, valid_cnn_logits, valid_acc

In [0]:
## Gets logits of the original model while training the substitute model 
def get_logits(sess,x):
  with tf.variable_scope(TOP_SCOPE): 
    batch = x[:BATCH_SHAPE]
    non_adv_output = get_logits_cnn(tf.cast(np.array(batch),tf.float32))
    for batch_num in range(1,int(len(x)/BATCH_SHAPE)): 
      print(batch_num)
      batch = valid_X[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
      non_adv_out = get_logits_cnn(tf.cast(np.array(batch),tf.float32))
      non_adv_output = tf.concat([non_adv_output,non_adv_out],axis=0)
    return sess.run(non_adv_output)
    

In [0]:
## Class for training cleverhans substitute model 
class ModelSubstitute(Model): 
  def __init__(self, scope, **kwargs):
    del kwargs
    Model.__init__(self, scope, nb_classes, locals())
    self.nb_filters = nb_filters
  
  def fprop(self, x, **kwargs):
    del kwargs
    with tf.variable_scope('model_s',reuse=tf.AUTO_REUSE):
      logits = get_sub_logits(x)
      print("GET LOGITS")
    return {self.O_LOGITS: logits,
            self.O_PROBS: tf.nn.softmax(logits=logits)}

## Trains the cleverhans substitute model 
def train_sub(sess, x, y, bbox_preds, x_sub, y_sub, nb_classes,
              nb_epochs_s, batch_size, learning_rate, data_aug, lmbda,
              aug_batch_size, rng,nchannels=1):
  with tf.variable_scope('model_s',reuse=tf.AUTO_REUSE):
    model_sub = ModelSubstitute('model_s', nb_classes)
    preds_sub = model_sub.get_logits(x)
    loss_sub = CrossEntropy(model_sub, smoothing=0)
    # Define the Jacobian symbolically using TensorFlow
    grads = jacobian_graph(preds_sub, x, nb_classes)
    # Train the substitute and augment dataset alternatively
    for rho in xrange(data_aug):
      print("Substitute training epoch #" + str(rho))
      train_params = {
          'nb_epochs': nb_epochs_s,
          'batch_size': batch_size,
          'learning_rate': learning_rate
      }
      print("TRAINING")
      train(sess, loss_sub,x_sub.astype(float), to_categorical(y_sub, nb_classes),
              init_all=False, args=train_params, rng=rng,
              var_list=model_sub.get_params())
      print("END_TRAINING")
      if rho < data_aug - 1:
        print("Augmenting substitute training data.")
        lmbda_coef = 2 * int(int(rho / 3) != 0) - 1
        x_sub = jacobian_augmentation(sess, x, x_sub, y_sub, grads,
                                      lmbda_coef * lmbda, aug_batch_size)
        print("Labeling substitute training data.")
        y_sub = np.hstack([y_sub, y_sub])
        x_sub_prev = x_sub[int(len(x_sub)/2):]
        eval_params = {'batch_size': batch_size}
        # bbox_val = batch_eval(sess, [x], [tf.convert_to_tensor(bbox_preds[:256])],
        #                       [x_sub_prev], batch_size=256)[0]
        bbox_val = get_logits(sess,x_sub_prev)
        print(len(bbox_val))               
        # Note here that we take the argmax because the adversary
        # only has access to the label (not the probabilities) output
        # by the black-box model

        y_sub[int(len(x_sub)/2):] = np.argmax(bbox_val, axis=1)
    with open('./DP-GRAPHS/sub_x','wb') as fp:
      pickle.dump(x_sub,fp)
    with open('./DP-GRAPHS/sub_y','wb') as fp:
      pickle.dump(y_sub,fp)
    return model_sub, preds_sub


In [0]:
## Defines and trains cnn blackbox model for generating the adversarial examples
def cnn_blackbox(nb_classes=NB_CLASSES,
                   batch_size=BATCH_SIZE, learning_rate=LEARNING_RATE,
                   nb_epochs=NB_EPOCHS, holdout=HOLDOUT, data_aug=DATA_AUG,
                   nb_epochs_s=NB_EPOCHS_S, lmbda=LMBDA,
                   aug_batch_size=AUG_BATCH_SIZE):
  # Dictionary used to keep track and return key accuracies
  accuracies = {}
  session = tf.Session()
  with session as sess:
    TOP_SCOPE = tf.get_variable_scope()
    ## Restore the original model for getting logits
    second_saver.restore(sess, './DP-GRAPHS/CNN_BASIC.ckpt')
    x_sub = valid_X[:holdout]
    y_sub = np.argmax(valid_labels[:holdout], axis=1)

    x_test = valid_X[holdout:]
    y_test = valid_labels[holdout:]

    x = tf.placeholder(tf.float32, shape=(BATCH_SIZE, CONT_FEATURE_SIZE))
    y = tf.placeholder(tf.float32, shape=(BATCH_SIZE, nb_classes))

    rng = np.random.RandomState([2019, 8, 6])
    print("Preparing the black-box model.")
    prep_bbox_out = prep_bbox(sess)
    model, bbox_preds, accuracies['bbox'] = prep_bbox_out
    MODEL = model
    BBOX_PREDS= bbox_preds
    ACCURACY =  accuracies['bbox']
    print("Training the substitute model.")
    train_sub_out = train_sub(sess, x, y, bbox_preds, x_sub, y_sub,
                              nb_classes, nb_epochs_s, batch_size,
                              learning_rate, data_aug, lmbda, aug_batch_size,
                              rng)
    model_sub, preds_sub = train_sub_out
    eval_params = {'batch_size': batch_size}
    acc = model_eval(sess, x, y, preds_sub, np.array(x_test), np.array(y_test), args=eval_params)
    accuracies['sub'] = acc

In [0]:
if CREATE:
  cnn_blackbox(nb_classes=2,batch_size=128,learning_rate=0.001,nb_epochs=10,holdout=512,data_aug=6)

In [0]:
## Reopen the synthetic substitute training model data
with open('./DP-GRAPHS/sub_x','rb') as fp:
  x_sub = pickle.load (fp)
with open('./DP-GRAPHS/sub_y','rb') as fp:
  y_sub = pickle.load(fp)

In [0]:
## Train substitute Model
with tf.device('/device:GPU:0'):
  sub_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y2_, logits=get_next_sub_logits(x2_)))
  sub_class_train_step = tf.train.AdamOptimizer(1e-3,name="sub-adam-encoder").minimize(sub_cross_entropy)
  sub_correct_prediction = tf.equal(tf.argmax(get_next_sub_logits(x2_), 1), tf.argmax(y2_, 1))
  sub_accuracy = tf.reduce_mean(tf.cast(sub_correct_prediction, tf.float32))

In [0]:
## Saver for reopening the substitute model 
saver = tf.train.Saver()

In [0]:
## Train the substitute model using the generated data
if CREATE:
  session = tf.Session(config=config)
  session.run(tf.global_variables_initializer())
  SIZE = 128
  EPOCHS = 10
  with session as sess:
    second_saver.restore(sess, './DP-GRAPHS/CNN_BASIC.ckpt')
    while SIZE < len(x_sub)*2:
      batches = x_sub[:SIZE]
      labels = y_labels[:SIZE]
      EPOCHS = EPOCHS+10
      for EPOCH in range(EPOCHS):
        train_acc = 0
        for batch_num in range(int(len(batches)/BATCH_SHAPE)):
          batch = batches[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
          batch_label = labels[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
          current_train_acc = sub_accuracy.eval(feed_dict={x2_:batch, y2_: batch_label})
          sub_class_train_step.run(feed_dict={x2_:batch, y2_: batch_label})
          train_acc += current_train_acc
        train_acc = float(train_acc)/(len(batches)/BATCH_SHAPE)
        print("EPOCH: " +str(EPOCH) +" Accuracy: "+ str(train_acc))
      SIZE= SIZE*2
    saver.save(sess, './DP-GRAPHS/USTC2-sub.ckpt')

In [0]:
## Tensor for getting the CNN loits 
cnn_logits = get_logits_cnn(x2_)

In [0]:
## Get valid malicious flows  ##
index = 0 
indices = []
for label in valid_labels:
  if label[0] == 1:
    indices.append(index)
  index+=1
valid_malicious_flows = []
for i in indices:
  valid_malicious_flows.append(valid_X[i])
print(len(valid_malicious_flows))

## Get benign flows ##
index = 0 
for label in valid_labels:
  if label[0] == 0:
    indices.append(index)
  index+=1
valid_benign_flows = []
for i in indices:
  valid_benign_flows.append(valid_X[i])
print(len(valid_benign_flows))

In [0]:
## Get malicious flows for creating adversarial examples 
NUM_SAMPLED = 17664
test_malicious_flows_index = np.random.choice(len(valid_malicious_flows), NUM_SAMPLED , replace=False)
test_malicious_flows = []
for index in test_malicious_flows_index:
  test_malicious_flows.append(valid_malicious_flows[index])
test_malicious_flows = np.array(test_malicious_flows).astype(float)

In [0]:
session = tf.Session(config=config)
session.run(tf.global_variables_initializer())
accuracy =1
with session as sess: 

  ## Restore the USTC substitute model 
  saver.restore(sess, './DP-GRAPHS/USTC2-sub.ckpt')
  with  tf.device('/device:GPU:0'):
    print("Classification Network Benign vs Malicious ")
    valid_acc = 0 

    ## Check the validation accuracy 
    for batch_num in range(int(len(valid_X)/BATCH_SHAPE)): 
      batch = valid_X[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
      batch_label = valid_labels[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
      valid_acc += cnn_accuracy.eval(feed_dict={x2_:batch, y2_: batch_label})
    print('Step %d, validation accuracy %g' % (batch_num , valid_acc/(int(len(valid_X)/BATCH_SHAPE))))     
    
    ## Get the the callable cleverhans model for creating the 
    ## adversarial model 
    nn_model = chm.CallableModelWrapper(get_next_sub_logits, 'logits')
    fgsm = SparseL1Descent(nn_model,sess =sess)
    fgsm_params = {'eps_iter':0.005,
            'eps': SCALE,
            'nb_iter': 100,                   
            'ord': 1,
                        'clip_min': 0,
                        'clip_max': 1.0,
                      'y_target':tf.convert_to_tensor(np.array([[0,1]]*128))}
    batch = test_malicious_flows[:BATCH_SHAPE]

    ## Gernerate the adversarial example 
    adv_x = fgsm.generate(tf.cast(np.array(batch),tf.float32), **fgsm_params)

    ## Ge al the adversrial examples
    for batch_num in range(1,int(len(test_malicious_flows)/BATCH_SHAPE)):
      print(batch_num)
      batch =test_malicious_flows[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
      batch_size = len(batch)
      adv_output = fgsm.generate(tf.cast(np.array(batch),tf.float32), **fgsm_params)
      adv_x  = tf.concat([adv_x, adv_output[:batch_size]],axis = 0)
    adversarial_output = sess.run(adv_x)

    print("GET SOFTMAX ADV EXAMPLES")
    ## Run to get softmax adversarial outputs ##
    batch = adversarial_output[:BATCH_SHAPE]
    adv_soft_output = tf.nn.softmax(cnn_logits.eval(feed_dict={x2_:batch}))
    for batch_num in range(1,int(len(adversarial_output)/BATCH_SHAPE)):
      print(batch_num)
      batch = adversarial_output[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
      adv_soft_out = tf.nn.softmax(cnn_logits.eval(feed_dict={x2_:batch}))
      adv_soft_output = tf.concat([adv_soft_output,adv_soft_out],axis = 0) 
    adv_soft_output = sess.run(adv_soft_output) 

    ## Get the final accuracies 
    accuracy = get_accuracy([[1,0]]*128,adv_soft_output[:18176])
    print('Test accuracy of oracle on adversarial examples generated '
          'using the substitute: ' + str(accuracy))
    print(accuracy)

In [0]:
FOLDER = 'USTC2-BasicBlackBox'
with open('./ADV-Example/' + FOLDER+ '/RESULTS'+SCALE_STRING+'.txt', 'w') as fp:
  fp.write(" Test accuracy of oracle on adversarial examples generated using the substitute: " +str(accuracy))
  fp.close()